
Multi-armed bandits is a rich, multi-disciplinary research area which receives attention from computer science, operations research, economics and statistics. It has been studied since (Thompson, 1933), with a big
surge of activity in the past 15-20 years


Other sources:
    * https://tor-lattimore.com/downloads/book/book.pdf by lattimore2020bandit by 
    * Intro to Multi-Armed Bandits slivkins2019introduction
@book{lattimore2020bandit,
  title={Bandit algorithms},
  author={Lattimore, Tor and Szepesv{\'a}ri, Csaba},
  year={2020},
  publisher={Cambridge University Press}
}
          
@article{slivkins2019introduction,
  title={Introduction to multi-armed bandits},
  author={Slivkins, Aleksandrs and others},
  journal={Foundations and Trends{\textregistered} in Machine Learning},
  volume={12},
  number={1-2},
  pages={1--286},
  year={2019},
  publisher={Now Publishers, Inc.}
}
    
@article{thompson1933likelihood,
  title={On the likelihood that one unknown probability exceeds another in view of the evidence of two samples},
  author={Thompson, William R},
  journal={Biometrika},
  volume={25},
  number={3-4},
  pages={285--294},
  year={1933},
  publisher={Oxford University Press}
}

# Multi-Armed Bandit Problem

## Introduction

The Multi-Armed Bandit (MAB) problem is a classical reinforcement learning problem that exemplifies the exploration-exploitation trade-off. The goal is to maximize the cumulative reward by choosing the best arm to pull at each time step.

## Formal Definition

A multi-armed bandit problem can be formally defined as follows:
- **Arms**: A set of $K$ arms, where each arm $i$ provides a reward from an unknown probability distribution $P_i$.
- **Time Steps**: The agent interacts with the bandit over a sequence of $T$ time steps.
- **Rewards**: At each time step $t$, the agent selects an arm $a_t$ and receives a reward $r_t$ drawn from the distribution $P_{a_t}$.

The objective is to maximize the expected cumulative reward over $T$ time steps.

POssible applications; 
a crowdsourcing platform can improve the assignment of tasks, workers and prices

## Key Concepts

### Exploration vs. Exploitation

- **Exploration**: Trying different arms to gather information about their reward distributions.
- **Exploitation**: Selecting the arm with the highest known reward based on the information gathered so far.

Balancing these two aspects is crucial for achieving optimal performance in the MAB problem.

### Regret

Regret measures the difference between the cumulative reward obtained by the optimal strategy and the reward obtained by the algorithm. It is defined as:

$$
R(T) = T \mu^* - \sum_{t=1}^T \mu_{a_t}
$$

where $\mu^*$ is the mean reward of the best arm, and $\mu_{a_t}$ is the mean reward of the arm chosen at time $t$.

## Strategies for Multi-Armed Bandit Problems

### 1. Epsilon-Greedy

The epsilon-greedy strategy is one of the simplest approaches:
- With probability $\epsilon$, select a random arm (exploration).
- With probability $1 - \epsilon$, select the arm with the highest average reward observed so far (exploitation).

In [1]:
import numpy as np

class EpsilonGreedy:
    def __init__(self, n_arms, epsilon):
        self.n_arms = n_arms
        self.epsilon = epsilon
        self.counts = np.zeros(n_arms)
        self.values = np.zeros(n_arms)
    
    def select_arm(self):
        if np.random.rand() < self.epsilon:
            return np.random.randint(0, self.n_arms)
        else:
            return np.argmax(self.values)
    
    def update(self, chosen_arm, reward):
        self.counts[chosen_arm] += 1
        n = self.counts[chosen_arm]
        value = self.values[chosen_arm]
        self.values[chosen_arm] = ((n - 1) / n) * value + (1 / n) * reward

# Example usage
n_arms = 10
epsilon = 0.1
n_rounds = 1000

bandit = EpsilonGreedy(n_arms, epsilon)
rewards = np.random.rand(n_arms)  # Simulated rewards for each arm

for _ in range(n_rounds):
    chosen_arm = bandit.select_arm()
    reward = rewards[chosen_arm]
    bandit.update(chosen_arm, reward)

print("Estimated values:", bandit.values)


Estimated values: [0.14915631 0.52210932 0.48625763 0.99539302 0.760989   0.1701503
 0.1915776  0.34115175 0.85734105 0.05478585]


### Uppder Confidence Bound (UCB)
The UCB algorithm selects the arm that maximizes the upper confidence bound of the reward estimate. For each arm $i$, the upper confidence bound is calculated as:
$$
UCB_i (t) = \hat{\mu}_i + \sqrt{\frac{2 \ln t}{n_i}}
$$

In [2]:
class UCB1:
    def __init__(self, n_arms):
        self.n_arms = n_arms
        self.counts = np.zeros(n_arms)
        self.values = np.zeros(n_arms)
    
    def select_arm(self):
        total_counts = np.sum(self.counts)
        if 0 in self.counts:
            return np.argmin(self.counts)
        else:
            ucb_values = self.values + np.sqrt((2 * np.log(total_counts)) / self.counts)
            return np.argmax(ucb_values)
    
    def update(self, chosen_arm, reward):
        self.counts[chosen_arm] += 1
        n = self.counts[chosen_arm]
        value = self.values[chosen_arm]
        self.values[chosen_arm] = ((n - 1) / n) * value + (1 / n) * reward

# Example usage
bandit = UCB1(n_arms)

for _ in range(n_rounds):
    chosen_arm = bandit.select_arm()
    reward = rewards[chosen_arm]
    bandit.update(chosen_arm, reward)

print("Estimated values:", bandit.values)


Estimated values: [0.14915631 0.52210932 0.48625763 0.99539302 0.760989   0.1701503
 0.1915776  0.34115175 0.85734105 0.05478585]


### Thompson Sampling
Thompson Sampling is a Bayesian approach to the MAB problem:

- Maintain a probability distribution (prior) over the possible reward distributions of each arm.
- At each time step, sample from these distributions (posterior) and select the arm with the highest sampled value.
- Update the posterior distributions based on the observed rewards.

In [3]:
class ThompsonSampling:
    def __init__(self, n_arms):
        self.n_arms = n_arms
        self.successes = np.zeros(n_arms)
        self.failures = np.zeros(n_arms)
    
    def select_arm(self):
        samples = np.random.beta(self.successes + 1, self.failures + 1)
        return np.argmax(samples)
    
    def update(self, chosen_arm, reward):
        if reward == 1:
            self.successes[chosen_arm] += 1
        else:
            self.failures[chosen_arm] += 1

# Example usage
bandit = ThompsonSampling(n_arms)

for _ in range(n_rounds):
    chosen_arm = bandit.select_arm()
    reward = np.random.binomial(1, rewards[chosen_arm])  # Simulated binary reward
    bandit.update(chosen_arm, reward)

print("Successes:", bandit.successes)
print("Failures:", bandit.failures)


Successes: [  1.   1.   0. 965.   0.   0.   0.   1.  13.   0.]
Failures: [1. 2. 2. 4. 2. 2. 1. 1. 3. 1.]


## Variants of the Multi-Armed Bandit Problem
### Contextual Bandits
In contextual bandits, the decision-making process is conditioned on the context $x_t$ available at each time step $t$. The objective is to learn a policy that maps contexts to arms, maximizing the cumulative reward.

In [4]:
class ContextualBandit:
    def __init__(self, n_arms, n_features):
        self.n_arms = n_arms
        self.n_features = n_features
        self.beta = np.zeros((n_arms, n_features))
    
    def select_arm(self, context):
        means = context @ self.beta.T
        return np.argmax(means)
    
    def update(self, chosen_arm, context, reward):
        x = context
        self.beta[chosen_arm] += (reward - x @ self.beta[chosen_arm]) * x

# Example usage
n_features = 5
contextual_bandit = ContextualBandit(n_arms, n_features)
contexts = np.random.rand(n_rounds, n_features)  # Simulated contexts

for t in range(n_rounds):
    context = contexts[t]
    chosen_arm = contextual_bandit.select_arm(context)
    reward = rewards[chosen_arm]  # Simulated reward based on context
    contextual_bandit.update(chosen_arm, context, reward)

print("Estimated beta values:", contextual_bandit.beta)


Estimated beta values: [[-4.18811054e-02 -5.76686108e-02 -9.55660940e-02 -6.14116128e-02
  -2.09012132e-01]
 [-2.69611909e-01 -2.78705183e-01 -1.79543465e-01 -9.26437616e-02
  -9.71837634e-02]
 [-3.33083324e-01 -2.52651385e-01 -3.19156163e-01 -6.83125569e-01
  -3.35290587e-01]
 [-3.00467737e-01 -1.54410670e-01 -3.40915593e-01  1.07832829e-02
  -1.07722033e-01]
 [-2.42237296e-01  1.37255424e-01 -8.63753225e-01 -8.76779119e-03
  -2.70387902e-01]
 [-2.81536735e-01 -1.38880891e-01 -8.68862001e-02 -8.15497878e-02
  -3.39912484e-01]
 [-2.11081451e-01 -1.25938685e-01 -6.50009732e-02 -7.97526458e-02
  -1.22911190e-01]
 [-2.30866679e-01 -2.33171158e-01  6.74073763e-02 -2.08244699e-01
  -1.70848785e-01]
 [-7.31335998e-01 -1.29179557e+00  5.37027553e-01  1.46193163e+00
   1.20663803e+00]
 [ 1.82195012e-01 -6.63277737e-03 -1.35120108e-03 -1.15987907e-02
   8.09158509e-02]]


### Non-Stationary Bandits
Non-stationary bandits consider scenarios where the reward distributions of the arms change over time. Algorithms for this variant need to adapt to these changes, often incorporating mechanisms for discounting past observations or detecting change points.

### Combinatorial Bandits
Combinatorial bandits involve selecting subsets of arms rather than a single arm at each time step. This variant is relevant in scenarios where the decision-maker must allocate resources across multiple options simultaneously.

## Applications
The multi-armed bandit problem has numerous real-world applications, including:

- Online Advertising: Selecting which ads to display to maximize click-through rates.
- Clinical Trials: Allocating treatments to patients to identify the most effective treatment.
- Recommendation Systems: Presenting items (e.g., movies, products) to users to maximize engagement.

## Conclusion
The Multi-Armed Bandit problem is a fundamental challenge in reinforcement learning and decision-making under uncertainty. By effectively balancing exploration and exploitation, various algorithms can achieve near-optimal performance in diverse settings. Understanding and applying these strategies can lead to significant improvements in fields ranging from online advertising to clinical trials and beyond.